In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import fiona
import h3pandas as h3
import networkx as nx
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [2]:
# Network Files
roads = gpd.read_file(r"H:\30\30555 - Palo Alto Southern Rail Crossing Study\GIS\python_access\Network_Development_workzone\python_outputs\draft_network_rds.shp")
nodes_r =gpd.read_file(r"H:\30\30555 - Palo Alto Southern Rail Crossing Study\GIS\python_access\Network_Development_workzone\python_outputs\draft_network_nodes.shp")

In [4]:
roads = roads.set_index(['u','v','key'])
nodes = nodes_r.set_index('osmid')

In [5]:
G = ox.graph_from_gdfs(nodes, roads)
G.remove_nodes_from(list(nx.isolates(G)))

In [7]:
# Prior Code with the Start and end locations for GPS

charleston = [37.414259, -122.118979, 'charl']
meadow = [37.417171, -122.123659, 'mead']
rail_cal = [37.429101, -122.142356,'call_under']
rail_cal2 = [37.429870, -122.141825,'call_under']
rail_sa = [37.406536, -122.106267,'san_under']

locs = [rail_cal,charleston,meadow,rail_sa,rail_cal2]

In [ ]:
# Add to locs the starting points for the new crossings for running access

In [8]:
# Set walking and biking speed
walk_sp = 3 / 60 * 5
bike_sp = 10 / 60 * 5

In [9]:
# Set testing distances
dist_walk = [walk_sp, walk_sp*2, walk_sp*3]
dist_bike = [bike_sp, bike_sp*2, bike_sp*3]

In [11]:
roads_record = pd.DataFrame()
nodes_record = pd.DataFrame()

for x in locs:
    point = ox.distance.nearest_nodes(G, X = x[1] , Y = x[0])
    name = x[2]

    for y in dist_walk:
        try:
            distance = y * 1609
            subgraph = nx.ego_graph(G, point, radius = distance, distance='length')
            access_pts, access_rds = ox.graph_to_gdfs(subgraph)
            access_pts['loc'] = x[2]
            access_rds['loc'] = x[2]
            access_pts['dist'] = y
            access_rds['dist'] = y
            access_pts['mode'] = 'walk'
            access_rds['mode'] = 'walk'
            access_rds['name'] = name
            roads_record = pd.concat([roads_record,access_rds])
            nodes_record = pd.concat([nodes_record,access_pts])
        except:
            continue

for x in locs:
    point = ox.distance.nearest_nodes(G, X = x[1] , Y = x[0])
    name = x[2]

    for y in dist_bike:
        try:
            distance = y * 1609
            subgraph = nx.ego_graph(G, point, radius = distance, distance='palo_length')
            access_pts, access_rds = ox.graph_to_gdfs(subgraph)
            access_pts['loc'] = x[2]
            access_rds['loc'] = x[2]
            access_pts['dist'] = y
            access_rds['dist'] = y
            access_pts['mode'] = 'bike_palo'
            access_rds['mode'] = 'bike_palo'
            access_rds['name'] = name
            roads_record = pd.concat([roads_record,access_rds])
            nodes_record = pd.concat([nodes_record,access_pts])
        except:
            continue

In [ ]:
# Resulting dataframe will have access from all points on either side of crossings
# Write loop to turn on and off different segments from the new crossings
# Loop will retain only shortest travel time segment.
# Highlight segments with reduced travel time with new crossing